In [1]:
#| default_exp datasets

# Loading Data

Formatting data for use with a new library can be a time consuming process. 
If you're using multiple libraries with different data requirements, you may find yourself reformatting the same data multiple times, storing multiple copies of the same data in different formats, and struggling to synchronize changes between them.
Rather than requiring users to reformat their data to fit the library, we we borrow an approach from [Pytorch](https://pytorch.org/).
Instead of requiring data be specified in a specific way, Pytorch provides an abstract Dataset class that can be extended to provide a common interface for loading data from a variety of sources. 
Instead of generating the dataset in a new format, the job becomes to provide functions retrieve data samples from the original source and return them in the appropriate format.

Here we outline this approach in more detail (often cribbing from the Pytorch documentation) and provide examples of how we have used it to load data in our own research.
More specific instructions about what information a Dataset should return to work with applicable functions our library will be provided in their specific documentation.

## The Dataset Class

In [2]:
#| code-summary: specify the abstract class for datasets

#| export
from abc import ABC, abstractmethod
from typing import List, Dict

class Dataset(ABC):
    """
    `Dataset` is an abstract class representing a dataset. Your custom dataset should inherit Dataset and override the following methods:

    - `__len__` so that len(dataset) returns the size of the dataset.
    - `__getitem__` to support the indexing such that `dataset[i]` can be used to get ith sample
    """

    @abstractmethod
    def __len__(self
                )->int: # number of samples in the dataset
        """
        Returns the number of samples in the dataset.
        """
        pass

    @abstractmethod
    def __getitem__(self, idx: int) -> List[Dict]:
        """
        Returns a sample from the dataset.
        """
        pass

As a toy example, the following class specifies the interface for a dataset that accesses a text file and treats each line as a separate sample to be represented as a Markdown object.

In [3]:
#| code-summary: specify a toy concrete class for text-based datasets

class TextDataset(Dataset):

    def __init__(self, dataset_path: str):
        with open(dataset_path, 'r') as f:
            self.data = f.read().split('\n')

    def __len__(self):
        return self.data.count('\n')
    
    def __getitem__(self, idx):
        return self.data[idx]

We can use it to consider lines of this project's README.

In [4]:
#| code-summary: demonstrate how to use the toy concrete class

readme_dataset = TextDataset('../../README.md')
print(len(readme_dataset))
print(readme_dataset[0][:50])

0
This project explores the potential of automatic n


## Senses Dataset

In [5]:
#| code-summary: specify the Senses dataset

#| export
import numpy as np
import chardet
import hdf5storage

class SensesDataset(Dataset):
    """
    Dataset class for the Senses dataset.
    """

    def __init__(self, hdf5_file_path, sense_pool_path):
        """
        Initialize the dataset.

        Args:
            hdf5_file_path (str): Path to the HDF5 file.
            sense_pool_path (str): Path to the sense pool file.
            transform (callable, optional): Optional transform to be applied
                on a sample. Defaults to None.
        """
        # Load the data from the specified HDF5 file
        # You can customize this part to load your data
        self.data = hdf5storage.read(path='/data', filename=hdf5_file_path)

        self.trial_count = len(self.data["subject"])
        self.trial_indices = np.arange(self.trial_count)
        with open(sense_pool_path, mode='rb') as f:
            raw_data = f.read()
            detected_encoding = chardet.detect(raw_data)['encoding']
            self.sense_pool = raw_data.decode(str(detected_encoding)).split('\n')

    def __len__(self):
        """
        Return the total number of samples in the dataset.

        Returns:
            int: Number of trials in the dataset.
        """
        return self.trial_count

    def __getitem__(self, idx):
        """
        Get a trial from the dataset at the specified index.

        Args:
            trial_idx (int): Index of the trial to retrieve.

        Returns:
            sample: The retrieved sample.
        """

        # pres_itemids selects the indices from sense_pool of the target items
        trials = []
        for trial_idx in np.atleast_1d(self.trial_indices[idx]):
            senses = self.data['pres_itemids'][trial_idx]
            target_items = [self.sense_pool[each-1].strip() for each in senses if each != 0]
            
            # response units are the segments of the transcript selected by raters and their spans in the transcript
            response_transcript = str(self.data['recall_transcript'][trial_idx][0])
            response_units = [str(each) for each in self.data['response_units'][trial_idx] if str(each) != '']

            # full response units include text *and* span representations contained in a dict
            response_start_spans = [each-1 for each in self.data["response_unit_start"][trial_idx] if each != 0]
            response_end_spans = [each-1 for each in self.data["response_unit_end"][trial_idx] if each != 0]
            full_response_units = [{'text': unit, 'spans':[(start_span, end_span)]} for unit, start_span, end_span in zip(
                response_units, response_start_spans, response_end_spans)]
            
            # matchings
            match_matrix = np.zeros((len(target_items), len(response_units)), dtype=bool)
            target_indices = np.array([each-1 for each in self.data['recalls'][trial_idx] if each != 0])

            if len(target_indices) > 0:
                match_matrix[target_indices, np.arange(len(response_units))] = True

            trials.append({
                'target_context': '', 
                'target_items': target_items, 
                'response_transcript': response_transcript, 
                'response_units': full_response_units, 
                'matches': match_matrix.tolist()})
            
        return trials[0] if len(trials) == 1 else trials

For trials defining this dataset, participants were cued with a word and asked to generate all the senses of that word that they could think of.
The dataset is organized as an HDF5 file paired with a text file containing the pool of senses that participants could generate.
    
Each item returned by this dataset is a dictionary with the following keys:

- `target_context`: the story text 
- `target_items`: a list of dictionaries, each representing a target item in the story. 
- `response_transcript`: the response transcript  
- `response_units`: a list of dictionaries, each representing a response unit in the response transcript.
- `matches`: a boolean matrix indicating which response units match which target items

Each dictionary specified by a target item or response unit has the following keys:  

- `text`: the text of the target item  
- `spans`: a list of tuples, each representing a span of the target item in the story text

In [6]:
#| code-summary: load the Senses dataset

import os

section_tag = 'base' # unique identifier for this variation of notebook parameters
output_dir = 'C:/Users/gunnj/workspace/response_sequencer/data/'

dataset = SensesDataset(os.path.join(output_dir, f'{section_tag}_senses.h5'), os.path.join(output_dir, f'{section_tag}_sense_pool.txt'))

dataset[10]['response_units']

[{'text': "Um, as a verb, it is, like, to stop living. Um, you, it's also, like, um,",
  'spans': [(0, 73)]},
 {'text': 'dice is the plural of die, so, like, a six-sided object with different number of dots on each side. Um,',
  'spans': [(74, 177)]},
 {'text': 'D-Y-E dye is like, ink to add color to something. Um. . .',
  'spans': [(178, 235)]}]

## SBS Narrative Recall Dataset

In [7]:
#| code-summary: specify the SBS narrative dataset

#| export
import numpy as np
import json
import os

class SBS_NarrativeDataset(Dataset):
    """
    Dataset class for the narrative free recall dataset provided by Sarah Brown-Schmidt's Conversation lab.
    """

    def __init__(self, data_directory):
        """
        Initializes the dataset.

        Args:
            data_directory (str): the directory containing the dataset files
        """

        # each response file represents a sample; these are named based on '{story_name}_{subject_id}_{iteration}'
        self.data_directory = data_directory
        self.text_directory = os.path.join(data_directory, 'texts')
        self.sequence_directory = os.path.join(data_directory, 'sequences', 'human')
        self.response_files = []
        self.stories = {}
        for path, _, files in os.walk(self.text_directory):
            for name in files:
                if name.count('_') == 2:
                    self.response_files.append(name)
                else:
                    with open(os.path.join(path, name), 'r', encoding='utf-8') as f:
                        self.stories[name[:-4]] = f.read()

        self.trial_indices = np.arange(len(self.response_files))

    def _retrieve_story_text(self, response_file_name):
        """
        Retrieves the story text for the specified response transcript.
        """
        story_name = response_file_name.split('_')[0]
        return self.stories[story_name]

    def _retrieve_response_text(self, response_file_name):
        """
        Retrieves the response text for the specified response transcript.
        """
        story_name = response_file_name.split('_')[0]
        with open(os.path.join(self.text_directory, story_name, response_file_name), 'r') as f:
            return f.read()
        
    def _retrieve_response_sequence(self, response_file_name):
        """
        Retrieves the response sequence coded for the specified response transcript.
        """
        story_name = response_file_name.split('_')[0]
        with open(os.path.join(self.sequence_directory, story_name, response_file_name[:-3]+'json'), 'r') as f:
            return json.load(f)
        
    def _prepare_match_matrix(self, response_sequence):
        """
        Prepares a match matrix based on the specified response sequence.
        """
        matchings = response_sequence['correspondences']
        match_matrix = np.zeros(
            (len(response_sequence['source_units']), len(response_sequence['response_units'])), dtype=bool)

        for response_index, matched_target in enumerate(matchings):
            if matched_target > -1:
                match_matrix[matched_target, response_index] = True
        return match_matrix.tolist()
    
    def _prepare_target_items(self, story_text, response_sequence):
        """
        Prepares a list of target items based on the specified response sequence and story text.
        """
        updated_unit_start = [story_text.find(unit) for unit in response_sequence['source_units']]
        
        for index, unit in enumerate(response_sequence['source_units']):
            assert(updated_unit_start[index] > -1)

        return [{'text': unit, 'spans':[(start, start+len(unit))]} for unit, start in zip(
            response_sequence['source_units'], updated_unit_start)]
    
    def _prepare_response_units(self, response_text, response_sequence):
        """
        Prepares a list of response units based on the specified response sequence and response text.
        """
        updated_text = [
            response_text[span[0]:span[1]].strip() for span in response_sequence['response_spans']]

        return [{'text': unit, 'spans':[(span[0], span[0]+len(unit))]} for unit, span in zip(
            updated_text, response_sequence['response_spans'])]

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.response_files)
    
    def __getitem__(self, idx):
        """
        Returns a sample from the dataset.
        """
        
        trials = []
        for trial_idx in np.atleast_1d(self.trial_indices[idx]):

            response_file_name = self.response_files[trial_idx]
            story_text = self._retrieve_story_text(response_file_name)
            response_text = self._retrieve_response_text(response_file_name)
            response_sequence = self._retrieve_response_sequence(response_file_name)

            trials.append({
                'target_context': story_text, 
                'target_items': self._prepare_target_items(story_text, response_sequence), 
                'response_transcript': response_text, 
                'response_units': self._prepare_response_units(response_text, response_sequence), 
                'matches': self._prepare_match_matrix(response_sequence)})
            
        return trials[0] if len(trials) == 1 else trials

In this dataset, participants perform free recall of a story they previously read.
The dataset is organized into a directory of text files and spreadsheets that variously represent 
encoded stories, participant responses, and coding decisions made by the lab. 

Each item returned by this Dataset is structured similarly to those returned by the Senses Dataset.

In [8]:
#| code-summary: load the SBS narrative dataset

data_directory = 'C:/Users/gunnj/compmempy/data/narrative'
dataset = SBS_NarrativeDataset(data_directory)

## JSON Dataset
To store sequences for downstream evaluation, we'll use the JSON format.

In [9]:
#| code-summary: specify a concrete concrete interface for JSON datasets

#| export
import json

class JSON_Dataset(Dataset):

    def __init__(self, file_path: str):
        """
        Initializes the dataset.

        Args:
            file_path (str): the path to the JSON file containing the dataset
        """
        self.file_path = file_path
        with open(file_path, 'r') as f:
            self.data = json.load(f)

    def __len__(self
                )->int: # number of samples in the dataset
        """
        Returns the number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx: int) -> List[Dict]:
        """
        Returns a sample from the dataset.

        Each sample is a dictionary containing:
        - 'trial': the index from the reference dataset that the sample was derived from
        - 'target_context': The string containing the context of the target items. (if applicable)
        - 'target_items': The list of target items
        - 'response_transcript': The input text
        - 'response_units': The list of response units, a dictionary of the form {'text': str, 'span' [(start, end)]}
        - 'matches': a list of lists representing a 2-D boolean matrix array of shape (len(target_items), len(response_units)) containing True if the target item matches the response unit at the corresponding index.
        """
        return self.data[idx]

In [10]:
#| code-summary: load a generated dataset
dataset = JSON_Dataset('../../data/generated_dataset.json')
dataset[1]['response_units']

[{'text': 'Mug can mean, um, like a, a glass with a, but with a handle on it, so like a mug that you drink out of.',
  'spans': [[0, 103]]},
 {'text': 'Mug can, can be a criminal act where a person mugs somebody.',
  'spans': [[104, 164]]},
 {'text': "Um, mug can also, uh, reference a person's head.",
  'spans': [[165, 213]]},
 {'text': '(silence)', 'spans': [[214, 223]]}]

In [11]:
#| echo: false
import nbdev; nbdev.nbdev_export()